In [ ]:
# === CELL 1: SETUP ===
!pip install -q yfinance

import numpy as np
import pandas as pd
import yfinance as yf
import json
import time
import random
from datetime import datetime
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

print("🧬 Evolution Optimizer Ready")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# === CELL 2: EVOLUTION CONFIG ===
# UPDATED with learnings from first evolution run!

EVOLUTION_CONFIG = {
    'population_size': 80,      # More diversity (was 50)
    'generations': 50,          # More time to explore (was 30)
    'mutation_rate': 0.4,       # Higher mutation to escape local optima (was 0.3)
    'crossover_rate': 0.6,      # Slightly less breeding (was 0.7)
    'elite_keep': 3,            # Keep fewer elites to allow more exploration (was 5)
    'tournament_size': 4,       # Smaller tournaments = more randomness (was 5)
    'restart_every': 15,        # NEW: Inject fresh random DNA every 15 gens
}

# === EVOLVED BEST PARAMETERS (from first run) ===
# Use these as the starting point for further evolution
EVOLVED_BASELINE = {
    'rsi_oversold': 21,
    'rsi_overbought': 76,
    'momentum_min': 4,
    'bounce_min': 8,
    'drawdown_trigger': -6,
    'profit_target_1': 14,
    'profit_target_2': 25,
    'stop_loss': -19,
    'trailing_stop': 11,
    'max_hold_days': 32,
    'position_size': 0.21,
    'max_positions': 11
}

# === PARAMETERS TO EVOLVE (wider ranges for exploration) ===
# Each parameter has: [min, max, evolved_best]
EVOLVABLE_PARAMS = {
    # Entry thresholds - WIDER RANGES
    'rsi_oversold': [15, 50, 21],           # RSI for dip buy (evolved found 21)
    'rsi_overbought': [60, 90, 76],         # RSI to avoid
    'momentum_min': [2, 20, 4],             # Min 5d momentum % (evolved found 4)
    'bounce_min': [2, 15, 8],               # Min bounce % (evolved found 8)
    'drawdown_trigger': [-20, -3, -6],      # 21d drawdown for dip
    
    # Exit thresholds - WIDER RANGES
    'profit_target_1': [8, 25, 14],         # First profit take (evolved found 14)
    'profit_target_2': [15, 50, 25],        # Second profit take
    'stop_loss': [-25, -8, -19],            # Stop loss % (evolved found -19 WIDE)
    'trailing_stop': [5, 20, 11],           # Trailing stop % (evolved found 11)
    'max_hold_days': [15, 90, 32],          # Max days to hold (evolved found 32)
    
    # Position sizing - CRITICAL for returns
    'position_size': [0.08, 0.35, 0.21],    # % per position (evolved found 21%)
    'max_positions': [5, 20, 11],           # Max concurrent
}

WATCHLIST = [
    'APLD', 'SERV', 'MRVL', 'NVDA', 'AMD', 'MU', 'CRDO',
    'SMR', 'OKLO', 'LEU', 'UUUU', 'CCJ',
    'HOOD', 'LUNR', 'SNOW',
    'IONQ', 'RGTI', 'QUBT',
    'TSLA', 'META', 'GOOGL',
    'SPY', 'QQQ',
    'RIVN', 'LYFT'
]

print("🧬 EVOLUTION CONFIG v2.0 (Post-Learning)")
print(f"   Population: {EVOLUTION_CONFIG['population_size']} (was 50)")
print(f"   Generations: {EVOLUTION_CONFIG['generations']} (was 30)")
print(f"   Mutation Rate: {EVOLUTION_CONFIG['mutation_rate']} (was 0.3)")
print(f"   Restart Every: {EVOLUTION_CONFIG['restart_every']} gens")
print(f"   Evolvable params: {len(EVOLVABLE_PARAMS)}")
print(f"   Tickers: {len(WATCHLIST)}")
print(f"\n📊 STARTING FROM EVOLVED BASELINE:")
print(f"   Previous best: +31.3% return, 71.1% win rate")

In [ ]:
# === CELL 3: LOAD DATA ===
print("📥 Loading data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

# Split 70/30 train/test
train_data = {}
test_data = {}
for ticker, df in data_dict.items():
    split = int(len(df) * 0.7)
    train_data[ticker] = df.iloc[:split].reset_index(drop=True)
    test_data[ticker] = df.iloc[split:].reset_index(drop=True)

print(f"\n✅ Train: {len(train_data[list(train_data.keys())[0]])} days")
print(f"✅ Test: {len(test_data[list(test_data.keys())[0]])} days")

In [ ]:
# === CELL 4: FEATURE ENGINE ===
def compute_features(df):
    df = df.copy()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    
    # Returns
    for p in [1, 5, 10, 21]:
        df[f'ret_{p}d'] = c.pct_change(p) * 100
    
    # EMAs
    for p in [8, 13, 21, 34, 55]:
        df[f'ema_{p}'] = c.ewm(span=p).mean()
    df['ema_8_rising'] = (df['ema_8'] > df['ema_8'].shift(3)).astype(float)
    
    # Ribbon
    df['ribbon_bullish'] = ((df['ema_8'] > df['ema_13']) & (df['ema_13'] > df['ema_21'])).astype(float)
    df['ribbon_range'] = (df[['ema_8','ema_13','ema_21','ema_34','ema_55']].max(axis=1) - 
                          df[['ema_8','ema_13','ema_21','ema_34','ema_55']].min(axis=1)) / c * 100
    df['ribbon_tight'] = (df['ribbon_range'] < 5).astype(float)
    
    # RSI
    delta = c.diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    df['rsi'] = 100 - (100 / (1 + gain / (loss + 1e-10)))
    
    # MACD
    df['macd'] = c.ewm(span=12).mean() - c.ewm(span=26).mean()
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_rising'] = (df['macd'] - df['macd_signal'] > (df['macd'] - df['macd_signal']).shift(1)).astype(float)
    
    # Volume
    df['vol_ratio'] = v / (v.rolling(20).mean() + 1)
    df['vol_spike'] = (df['vol_ratio'] > 2).astype(float)
    
    # Momentum
    df['mom_5d'] = c.pct_change(5) * 100
    
    # Bounce
    df['low_5d'] = l.rolling(5).min()
    df['bounce'] = (c / (df['low_5d'] + 1e-10) - 1) * 100
    df['bounce_signal'] = ((df['bounce'] > 3) & (df['ema_8_rising'] > 0)).astype(float)
    
    # Trend
    df['trend_align'] = (np.sign(df['ret_5d']) + np.sign(df['ret_10d']) + np.sign(df['ret_21d'])) / 3
    
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill().fillna(0)
    return df

print("🧠 Computing features...")
train_features = {t: compute_features(df) for t, df in train_data.items()}
test_features = {t: compute_features(df) for t, df in test_data.items()}
print("✅ Features ready")

In [ ]:
# === CELL 5: DNA CLASS (Strategy Genome) ===

class StrategyDNA:
    """A trading strategy's genetic code"""
    
    def __init__(self, params=None):
        if params:
            self.params = params
        else:
            # Random initialization
            self.params = {}
            for name, (min_v, max_v, _) in EVOLVABLE_PARAMS.items():
                if isinstance(min_v, int):
                    self.params[name] = random.randint(min_v, max_v)
                else:
                    self.params[name] = random.uniform(min_v, max_v)
        
        self.fitness = 0
        self.test_fitness = 0
    
    def mutate(self):
        """Randomly mutate one parameter"""
        param = random.choice(list(self.params.keys()))
        min_v, max_v, _ = EVOLVABLE_PARAMS[param]
        
        # Mutation: adjust by 10-30%
        current = self.params[param]
        range_size = max_v - min_v
        mutation = random.uniform(-0.3, 0.3) * range_size
        new_val = current + mutation
        
        # Clamp to bounds
        new_val = max(min_v, min(max_v, new_val))
        
        if isinstance(min_v, int):
            new_val = int(new_val)
        
        self.params[param] = new_val
    
    @staticmethod
    def crossover(parent1, parent2):
        """Breed two strategies"""
        child_params = {}
        for param in parent1.params:
            # 50% chance from each parent, with some blending
            if random.random() < 0.5:
                child_params[param] = parent1.params[param]
            else:
                child_params[param] = parent2.params[param]
            
            # 20% chance to blend
            if random.random() < 0.2:
                blend = (parent1.params[param] + parent2.params[param]) / 2
                min_v, max_v, _ = EVOLVABLE_PARAMS[param]
                if isinstance(min_v, int):
                    blend = int(blend)
                child_params[param] = blend
        
        return StrategyDNA(child_params)
    
    def __repr__(self):
        return f"DNA(fitness={self.fitness:.1f})"

print("✅ DNA class ready")
# Test
test_dna = StrategyDNA()
print(f"   Sample DNA: {test_dna.params}")

In [ ]:
# === CELL 6: FITNESS EVALUATOR ===

def evaluate_fitness(dna, features_dict, data_dict):
    """Run backtest and return fitness score"""
    p = dna.params
    balance = 100000
    positions = {}
    trades = []
    history = [balance]
    
    tickers = list(features_dict.keys())
    min_len = min(len(df) for df in data_dict.values())
    
    for day in range(60, min_len - 1):
        prices = {}
        
        # Update positions
        for t in list(positions.keys()):
            if t in data_dict and day < len(data_dict[t]):
                price = float(data_dict[t]['Close'].iloc[day])
                prices[t] = price
                positions[t]['days'] += 1
                if price > positions[t]['max']:
                    positions[t]['max'] = price
        
        for ticker in tickers:
            if day >= len(features_dict[ticker]):
                continue
            
            df = features_dict[ticker]
            price = float(data_dict[ticker]['Close'].iloc[day])
            prices[ticker] = price
            
            rsi = float(df['rsi'].iloc[day])
            mom = float(df['mom_5d'].iloc[day])
            ret_21d = float(df['ret_21d'].iloc[day])
            ribbon_bull = float(df['ribbon_bullish'].iloc[day])
            macd_rising = float(df['macd_rising'].iloc[day])
            bounce = float(df['bounce'].iloc[day])
            bounce_sig = float(df['bounce_signal'].iloc[day])
            trend = float(df['trend_align'].iloc[day])
            
            # SELL
            if ticker in positions:
                pos = positions[ticker]
                pnl = (price / pos['entry'] - 1) * 100
                from_max = (price / pos['max'] - 1) * 100
                
                sell = False
                if pnl >= p['profit_target_2']:
                    sell = True
                elif pnl >= p['profit_target_1'] and rsi > p['rsi_overbought']:
                    sell = True
                elif pnl <= p['stop_loss']:
                    sell = True
                elif pnl > 15 and from_max < -p['trailing_stop']:
                    sell = True
                elif pos['days'] > p['max_hold_days'] and pnl > 0:
                    sell = True
                
                if sell:
                    balance += pos['shares'] * price
                    trades.append({'pnl': pnl})
                    del positions[ticker]
            
            # BUY
            else:
                if len(positions) >= int(p['max_positions']):
                    continue
                
                pv = balance + sum(positions[t]['shares'] * prices.get(t, 0) for t in positions)
                if balance / pv < 0.1:
                    continue
                
                # Check evolved entry conditions
                buy = False
                
                # Dip buy (evolved thresholds)
                if rsi < p['rsi_oversold'] and mom < -3:
                    buy = True
                
                # Bounce (evolved)
                if bounce > p['bounce_min'] and macd_rising > 0:
                    buy = True
                
                # Nuclear dip (evolved)
                if ret_21d < p['drawdown_trigger'] and macd_rising > 0:
                    buy = True
                
                # Momentum (evolved)
                if mom > p['momentum_min'] and macd_rising > 0 and bounce_sig > 0:
                    buy = True
                
                # Trend continuation (evolved)
                if trend > 0.5 and ribbon_bull > 0 and p['rsi_oversold'] < rsi < p['rsi_overbought']:
                    buy = True
                
                if buy:
                    shares = int(balance * p['position_size'] / price)
                    if shares > 0:
                        balance -= shares * price
                        positions[ticker] = {
                            'shares': shares,
                            'entry': price,
                            'max': price,
                            'days': 0
                        }
        
        pv = balance + sum(positions[t]['shares'] * prices.get(t, 0) for t in positions)
        history.append(pv)
    
    # Liquidate
    for ticker, pos in positions.items():
        if ticker in data_dict:
            price = float(data_dict[ticker]['Close'].iloc[-1])
            balance += pos['shares'] * price
            trades.append({'pnl': (price / pos['entry'] - 1) * 100})
    
    # Calculate fitness
    total_return = (balance / 100000 - 1) * 100
    winners = len([t for t in trades if t['pnl'] > 0])
    win_rate = winners / max(len(trades), 1)
    
    # Sharpe
    returns = np.diff(history) / (np.array(history[:-1]) + 1e-10)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-10) * np.sqrt(252)
    
    # Max drawdown
    peak = np.maximum.accumulate(history)
    dd = (np.array(history) - peak) / (peak + 1e-10)
    max_dd = np.min(dd)
    
    # FITNESS = Return * WinRate * Sharpe - Penalty for drawdown
    fitness = total_return * (0.5 + win_rate) * max(sharpe, 0.1) / (1 + abs(max_dd))
    
    return fitness, total_return, win_rate * 100, sharpe, max_dd * 100

print("✅ Fitness evaluator ready")

In [ ]:
# === CELL 7: EVOLUTION ENGINE ===

def evolve_population(population, features, data):
    """One generation of evolution"""
    
    # Evaluate all
    for dna in population:
        dna.fitness, _, _, _, _ = evaluate_fitness(dna, features, data)
    
    # Sort by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    # Keep elite
    new_pop = population[:EVOLUTION_CONFIG['elite_keep']]
    
    # Fill rest with offspring
    while len(new_pop) < EVOLUTION_CONFIG['population_size']:
        # Tournament selection
        def tournament():
            contestants = random.sample(population[:20], EVOLUTION_CONFIG['tournament_size'])
            return max(contestants, key=lambda x: x.fitness)
        
        parent1 = tournament()
        parent2 = tournament()
        
        # Crossover
        if random.random() < EVOLUTION_CONFIG['crossover_rate']:
            child = StrategyDNA.crossover(parent1, parent2)
        else:
            child = StrategyDNA(deepcopy(parent1.params))
        
        # Mutation
        if random.random() < EVOLUTION_CONFIG['mutation_rate']:
            child.mutate()
        
        new_pop.append(child)
    
    return new_pop

print("✅ Evolution engine ready")

In [ ]:
# === CELL 8: RUN EVOLUTION v2.0 ===
print("=" * 70)
print("🧬 STARTING EVOLUTION v2.0 (with restarts)")
print("=" * 70)

# Initialize population
population = [StrategyDNA() for _ in range(EVOLUTION_CONFIG['population_size'])]

# Add evolved baseline as first member (start from best known)
population[0] = StrategyDNA(EVOLVED_BASELINE.copy())

# Add human baseline too for comparison
human_params = {name: best for name, (_, _, best) in EVOLVABLE_PARAMS.items()}
population[1] = StrategyDNA(human_params)

best_ever = None
history = []
stagnation_count = 0
last_best_fitness = 0

start_time = time.time()

for gen in range(EVOLUTION_CONFIG['generations']):
    # Evolve on TRAINING data only
    population = evolve_population(population, train_features, train_data)
    
    best = population[0]
    avg_fitness = np.mean([d.fitness for d in population[:10]])
    
    # Track best ever
    if best_ever is None or best.fitness > best_ever.fitness:
        best_ever = StrategyDNA(deepcopy(best.params))
        best_ever.fitness = best.fitness
        stagnation_count = 0
    else:
        stagnation_count += 1
    
    history.append({
        'gen': gen,
        'best_fitness': best.fitness,
        'avg_fitness': avg_fitness
    })
    
    # RESTART: Inject fresh DNA if stagnating
    restart_every = EVOLUTION_CONFIG.get('restart_every', 15)
    if gen > 0 and gen % restart_every == 0:
        # Replace bottom 30% with fresh random DNA
        num_replace = int(EVOLUTION_CONFIG['population_size'] * 0.3)
        for i in range(-num_replace, 0):
            population[i] = StrategyDNA()  # Fresh random
        print(f"   🔄 Gen {gen}: Injected {num_replace} fresh random strategies")
    
    if gen % 5 == 0:
        elapsed = time.time() - start_time
        stag_warn = "⚠️ STAGNANT" if stagnation_count > 10 else ""
        print(f"Gen {gen:3d} | Best: {best.fitness:8.1f} | Avg Top10: {avg_fitness:8.1f} | {elapsed:.0f}s {stag_warn}")

print("-" * 70)
print(f"\n🏆 EVOLUTION v2.0 COMPLETE in {time.time()-start_time:.0f}s")
print(f"   Best Fitness: {best_ever.fitness:.1f}")
print(f"   Generations without improvement: {stagnation_count}")

In [ ]:
# === CELL 9: TEST BEST ON UNSEEN DATA ===
print("=" * 70)
print("🧪 TESTING EVOLVED STRATEGY ON UNSEEN DATA")
print("=" * 70)

# Test the evolved best
evolved_fitness, evolved_ret, evolved_wr, evolved_sharpe, evolved_dd = evaluate_fitness(
    best_ever, test_features, test_data
)

# Test human baseline
human_dna = StrategyDNA(human_params)
human_fitness, human_ret, human_wr, human_sharpe, human_dd = evaluate_fitness(
    human_dna, test_features, test_data
)

print(f"\n{'Metric':<20} {'Human':>15} {'Evolved':>15} {'Winner':>10}")
print("-" * 60)
print(f"{'Return':<20} {human_ret:>+14.1f}% {evolved_ret:>+14.1f}% {'🧬' if evolved_ret > human_ret else '👤'}")
print(f"{'Win Rate':<20} {human_wr:>14.0f}% {evolved_wr:>14.0f}% {'🧬' if evolved_wr > human_wr else '👤'}")
print(f"{'Sharpe Ratio':<20} {human_sharpe:>15.2f} {evolved_sharpe:>15.2f} {'🧬' if evolved_sharpe > human_sharpe else '👤'}")
print(f"{'Max Drawdown':<20} {human_dd:>14.1f}% {evolved_dd:>14.1f}% {'🧬' if evolved_dd > human_dd else '👤'}")
print(f"{'Fitness':<20} {human_fitness:>15.1f} {evolved_fitness:>15.1f} {'🧬' if evolved_fitness > human_fitness else '👤'}")

improvement = (evolved_ret - human_ret) / max(abs(human_ret), 1) * 100
print(f"\n📈 Evolution improved returns by {improvement:+.1f}%")

In [ ]:
# === CELL 10: EVOLVED PARAMETERS ===
print("=" * 70)
print("🧬 EVOLVED OPTIMAL PARAMETERS")
print("=" * 70)

print(f"\n{'Parameter':<25} {'Human':>12} {'Evolved':>12} {'Change':>12}")
print("-" * 65)

for param in EVOLVABLE_PARAMS:
    human_val = human_params[param]
    evolved_val = best_ever.params[param]
    
    if isinstance(human_val, float):
        change = (evolved_val - human_val) / max(abs(human_val), 0.01) * 100
        print(f"{param:<25} {human_val:>12.2f} {evolved_val:>12.2f} {change:>+11.1f}%")
    else:
        change = evolved_val - human_val
        print(f"{param:<25} {human_val:>12} {evolved_val:>12} {change:>+12}")

print("\n" + "=" * 70)
print("📋 COPY THESE EVOLVED SETTINGS:")
print("=" * 70)
print(json.dumps(best_ever.params, indent=2))

In [ ]:
# === CELL 11: SAVE RESULTS ===
results = {
    'generated_at': datetime.now().isoformat(),
    'evolution_config': EVOLUTION_CONFIG,
    'generations_run': EVOLUTION_CONFIG['generations'],
    'human_baseline': {
        'params': human_params,
        'test_return': human_ret,
        'test_win_rate': human_wr,
        'test_sharpe': human_sharpe,
        'test_max_dd': human_dd
    },
    'evolved_best': {
        'params': best_ever.params,
        'test_return': evolved_ret,
        'test_win_rate': evolved_wr,
        'test_sharpe': evolved_sharpe,
        'test_max_dd': evolved_dd,
        'fitness': best_ever.fitness
    },
    'improvement': {
        'return_pct': evolved_ret - human_ret,
        'win_rate_pct': evolved_wr - human_wr,
        'sharpe_diff': evolved_sharpe - human_sharpe
    },
    'evolution_history': history
}

with open('evolution_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ Results saved to evolution_results.json")

try:
    from google.colab import files
    files.download('evolution_results.json')
    print("📥 Download started!")
except:
    print("(Not in Colab - file saved locally)")

In [ ]:
# === CELL 12: RECOMMENDATIONS ===
print("=" * 70)
print("📋 FINAL RECOMMENDATIONS")
print("=" * 70)

if evolved_ret > human_ret * 1.1:  # 10% better
    print("\n🧬 USE EVOLVED PARAMETERS!")
    print("   The AI found significantly better settings.")
    rec = best_ever.params
else:
    print("\n👤 STICK WITH HUMAN PARAMETERS (or blend)")
    print("   Evolution didn't find major improvements.")
    rec = human_params

print("\n🎯 RECOMMENDED SETTINGS FOR YOUR TRADING:")
print("-" * 50)
print(f"   RSI Oversold: < {rec['rsi_oversold']:.0f}")
print(f"   RSI Overbought: > {rec['rsi_overbought']:.0f}")
print(f"   Momentum Entry: > {rec['momentum_min']:.0f}%")
print(f"   Bounce Entry: > {rec['bounce_min']:.1f}%")
print(f"   Dip Trigger (21d): < {rec['drawdown_trigger']:.0f}%")
print(f"   Profit Target 1: {rec['profit_target_1']:.0f}%")
print(f"   Profit Target 2: {rec['profit_target_2']:.0f}%")
print(f"   Stop Loss: {rec['stop_loss']:.0f}%")
print(f"   Trailing Stop: {rec['trailing_stop']:.0f}%")
print(f"   Max Hold Days: {rec['max_hold_days']:.0f}")
print(f"   Position Size: {rec['position_size']*100:.0f}%")
print(f"   Max Positions: {int(rec['max_positions'])}")